In [1]:
import os
import re
import pickle
import json
import gzip
import multiprocessing as mp
import nltk
from nltk.corpus import words

In [2]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /home/liufengyuan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [1]:
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Automotive.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Books.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_CDs_and_Vinyl.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Cell_Phones_and_Accessories.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Clothing_Shoes_and_Jewelry.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Digital_Music.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Electronics.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Grocery_and_Gourmet_Food.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Home_and_Kitchen.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Industrial_and_Scientific.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Kindle_Store.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Movies_and_TV.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Office_Products.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Patio_Lawn_and_Garden.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Pet_Supplies.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Sports_and_Outdoors.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Tools_and_Home_Improvement.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Toys_and_Games.json.gz --no-check-certificate

--2023-01-14 03:17:18--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 33023658 (31M) [application/x-gzip]
Saving to: ‘/data/liufengyuan/NLPinFinance/Data/meta_AMAZON_FASHION.json.gz’

meta_AMAZON_FASHION 100%[===================>]  31.49M  8.14MB/s    in 3.9s    

2023-01-14 03:17:23 (8.14 MB/s) - ‘/data/liufengyuan/NLPinFinance/Data/meta_AMAZON_FASHION.json.gz’ saved [33023658/33023658]

--2023-01-14 03:17:23--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.

In [39]:
data = []
with gzip.open('/data/liufengyuan/NLPinFinance/Data/meta_AMAZON_FASHION.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [17]:
print(len(data))

186637


In [24]:
count = 0
for i in range(20):
    if 'feature' in data[i].keys():
        count += 1

In [25]:
count

14

In [21]:
print(data[0].keys())

dict_keys(['title', 'brand', 'feature', 'rank', 'date', 'asin', 'imageURL', 'imageURLHighRes'])


In [22]:
print(data[1].keys())

dict_keys(['title', 'rank', 'date', 'asin', 'imageURL', 'imageURLHighRes'])


In [41]:
f_save = open('/data/liufengyuan/NLPinFinance/Filtered Data/AMAZON_FASHION.pkl', 'rb')
file_read = pickle.load(f_save)
f_save.close()

In [50]:
output = file_read[10]
output

{'asin': 'B0000AWYXK',
 'title': '14k White Gold 8mm Half Round Plain Wedding Band',
 'feature': ['Package Dimensions:\n                    \n2 x 2 x 1.9 inches',
  'Shipping Weight:\n                    \n3.2 ounces'],
 'description': ['With its smooth surface and high polish, this white gold wedding band is a stylish start to any marriage. The width of the ring is slightly rounded and the edges are straight, making the band appear more substantial while allowing it to sit comfortably. 14k gold is a durable fine metal, meaning this wedding band will last as long as your love.',
  '',
  '',
  '',
  'Discover the Amazon Curated Collection of fine and fashion jewelry. The expansive selection of high-quality jewelry featured in the Amazon Curated Collection offers everyday values that range from precious gemstone and diamond pieces to the latest fashion designs.',
  'This hand-selected assortment features an array of diamond, gemstone and precious metal designs that receive a quality insp

In [27]:
results = re.compile(r'#[]', re.S)
results2 = re.compile(r'<[|\n;#_\'(){}*-<>a-zA-Z0-9.?/&=:    " ]*', re.S)

In [9]:
# sentence = output['title']
# output = re.findall(pat, sentence.lower())
wordlist = {}
temp_output = []
pat = '[a-zA-Z]+'
features = output['feature']
for fea in features:
    #print(fea)
    #fea = re.sub(results, '', fea)
    fea = re.sub(results2, '', fea)
    print(fea)
    temp = re.findall(pat, fea)
    print(temp)
    temp_output.extend(temp)

Shimmer Anne Shine Clip On Costume/Halloween Cat Ears
['Shimmer', 'Anne', 'Shine', 'Clip', 'On', 'Costume', 'Halloween', 'Cat', 'Ears']
You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
['You', 'can', 'return', 'this', 'item', 'for', 'any', 'reason', 'and', 'get', 'a', 'full', 'refund', 'no', 'shipping', 'charges', 'The', 'item', 'must', 'be', 'returned', 'in', 'new', 'and', 'unused', 'condition']
Read the full returns policy
['Read', 'the', 'full', 'returns', 'policy']
Go to Your Orders to start the return
['Go', 'to', 'Your', 'Orders', 'to', 'start', 'the', 'return']
Print the return shipping label
['Print', 'the', 'return', 'shipping', 'label']
Ship it!
['Ship', 'it']
Package Dimensions:
                    
3.8 x 3 x 0.8 inches
['Package', 'Dimensions', 'x', 'x', 'inches']
Shipping Weight:
                    
0.64 ounces (View shipping rates and policies)
['Shipping', 'Weight', 'ounces', 'View

In [4]:
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)

In [4]:
import re

# 定义tokenize方法：
def tokenize(text):
    # 定义分割符，该变量作为超参数，可以根据情况删减 <[|\n;#_\'(){}*-<>a-zA-Z0-9.?/&=:
    filters = ['\|', '\?', '\{', '\}', '-', '\$', '\'', '\"', '\+', '\*', '!', '', '', ';', ':', '；','#', '%', '&', '\t', '\n', '\x97', '\x96', ',', '\(', '\)','\.']

    # 运用正则进行分词:
    # 将网页内容如<>以及分隔符全部替代为空格
    text = re.sub("<.*?>", " ", text, flags=re.S)
    # 将n't转换为not
    text = re.sub("n't", " not", text, flags=re.S)
    # 将感叹号单独分割出来
    #text = re.sub("(\w)!", "\g<1> !", text, flags=re.S)
    # 去掉一些特殊情况
    text = re.sub("\[download PDF version\]", " ", text, flags=re.S)
    text = re.sub("(View shipping rates and policies)", " ", text, flags=re.S)
    # 将分隔符全部替代为空格
    text = re.sub("|".join(filters), " ", text, flags=re.S)
    #text = emoji_pattern.sub(r'', text)
    # 运用strip()返回移除字符串头尾指定的字符（默认为空格或换行符）生成的新字符串:并转换为小写：
    result = [i.strip().lower() for i in text.split()]
    return result

In [ ]:
def tokenize(text):
    # 运用正则进行分词:
    # 将网页内容如<>以及分隔符全部替代为空格
    text = re.sub("<.*?>", " ", text, flags=re.S)
    # 将n't转换为not
    text = re.sub("n't", " not", text, flags=re.S)
    # 将感叹号单独分割出来
    #text = re.sub("(\w)!", "\g<1> !", text, flags=re.S)
    # 去掉一些特殊情况
    text = re.sub("\[download PDF version\]", " ", text, flags=re.S)
    text = re.sub("(View shipping rates and policies)", " ", text, flags=re.S)
    # 将分隔符全部替代为空格
    #text = emoji_pattern.sub(r'', text)
    # 运用strip()返回移除字符串头尾指定的字符（默认为空格或换行符）生成的新字符串:并转换为小写：
    result = [i.strip().lower() for i in text.split()]
    return result

In [10]:
features = output['feature']
for fea in features:
    #print(fea)
    #fea = re.sub(results, '', fea)
    #fea = re.sub(results2, '', fea)
    print(fea)
    temp = tokenize(fea)
    print(temp)

Shimmer Anne Shine Clip On Costume/Halloween Cat Ears
['shimmer', 'anne', 'shine', 'clip', 'on', 'costume/halloween', 'cat', 'ears']
You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
['you', 'can', 'return', 'this', 'item', 'for', 'any', 'reason', 'and', 'get', 'a', 'full', 'refund', 'no', 'shipping', 'charges', 'the', 'item', 'must', 'be', 'returned', 'in', 'new', 'and', 'unused', 'condition']
Read the full returns policy
['read', 'the', 'full', 'returns', 'policy']
Go to Your Orders to start the return
['go', 'to', 'your', 'orders', 'to', 'start', 'the', 'return']
Print the return shipping label
['print', 'the', 'return', 'shipping', 'label']
Ship it!
['ship', 'it', '!']
Package Dimensions:
                    
3.8 x 3 x 0.8 inches
['package', 'dimensions', '3', '8', 'x', '3', 'x', '0', '8', 'inches']
Shipping Weight:
                    
0.64 ounces (View shipping rates and policies)
['shipping'

In [115]:
filenames = os.listdir(r'/data/liufengyuan/NLPinFinance/Filtered Data')
wordlist = {}
pat = '[a-zA-Z]+'
total = 0
out = False
for name in filenames:
    f_save = open('/data/liufengyuan/NLPinFinance/Filtered Data/' + name, 'rb')
    print(name)
    file_read = pickle.load(f_save)
    f_save.close()
    for item in file_read:
        total += 1
        sentence = item['title']
        #print(sentence)
        #output = re.findall(pat, sentence.lower())
        #sentence = re.sub(results2, '', sentence)
        #output = tokenize(sentence.lower())
        output = re.findall(pat, sentence.lower())
        #output = []
        features = item['feature']
        for fea in features:
        #    if '#A' in fea:
        #        print(fea)
        #        out = True
        #        break
            fea = re.sub(results2, '', fea)
            #output.extend(tokenize(fea.lower()))
            output.extend(re.findall(pat, fea.lower()))
        for word in output:
            if word in wordlist.keys():
                wordlist[word] += 1
            else:
                wordlist[word] = 1
        #if out == True:
        #    out = False
        #    break
    #break
    print(total)
wordlist

Cell_Phones_and_Accessories.pkl
282198
Office_Products.pkl
417298
Movies_and_TV.pkl
417351
Patio_Lawn_and_Garden.pkl
529729
Clothing_Shoes_and_Jewelry.pkl
2043156
Electronics.pkl
2463881
Industrial_and_Scientific.pkl
2489700
Kindle_Store.pkl
2489700
Digital_Music.pkl
2489723
Home_and_Kitchen.pkl
3026039
Sports_and_Outdoors.pkl
3391802
CDs_and_Vinyl.pkl
3391873
Books.pkl
3392839
Arts_Crafts_and_Sewing.pkl
3505092
Toys_and_Games.pkl
3747531
Grocery_and_Gourmet_Food.pkl
3755740
AMAZON_FASHION.pkl
3764050
Pet_Supplies.pkl
3872506
Automotive.pkl
4291352
Tools_and_Home_Improvement.pkl
4527550


{'juicy': 2425,
 'couture': 5996,
 'durable': 386393,
 'layers': 24316,
 'crest': 3382,
 'case': 920178,
 'for': 5164854,
 'iphone': 333934,
 'hot': 92658,
 'pink': 150148,
 'free': 564159,
 'screen': 399438,
 'protector': 170327,
 'lightweight': 202922,
 'polycarbonate': 37521,
 'hard': 211962,
 'shell': 96690,
 'two': 294890,
 'piece': 227982,
 'snap': 116891,
 'together': 35636,
 'design': 740478,
 'full': 601355,
 'access': 178057,
 'to': 4362383,
 'all': 893512,
 'controls': 47226,
 'hi': 27417,
 'res': 12744,
 'image': 647888,
 'on': 1540279,
 'silver': 380863,
 'elegant': 84555,
 'butterfly': 26006,
 'foot': 73719,
 'ankle': 42748,
 'chain': 131563,
 'summer': 47431,
 'beach': 59584,
 'bracelet': 114945,
 'evil': 5234,
 'eye': 54287,
 'charm': 74027,
 'item': 485706,
 'is': 1767038,
 'brand': 332590,
 'new': 936440,
 'see': 128896,
 'description': 47592,
 'details': 66014,
 'we': 371118,
 'ship': 166408,
 'as': 568533,
 'fast': 116961,
 'can': 740392,
 'but': 123917,
 'please': 

In [ ]:
filenames = os.listdir(r'/data/liufengyuan/NLPinFinance/Filtered Data')
wordlist_title = {}
pat = '[a-zA-Z]+'
total = 0
out = False
for name in filenames:
    f_save = open('/data/liufengyuan/NLPinFinance/Filtered Data/' + name, 'rb')
    print(name)
    file_read = pickle.load(f_save)
    f_save.close()
    for item in file_read:
        total += 1
        sentence = item['title']
        #print(sentence)
        #output = re.findall(pat, sentence.lower())
        #sentence = re.sub(results2, '', sentence)
        #output = tokenize(sentence.lower())
        output = re.findall(pat, sentence.lower())
        #output = []
        #features = item['feature']
        #for fea in features:
        #    if '#A' in fea:
        #        print(fea)
        #        out = True
        #        break
        #    fea = re.sub(results2, '', fea)
            #output.extend(tokenize(fea.lower()))
        #    output.extend(re.findall(pat, fea))
        for word in output:
            if word in wordlist_title.keys():
                wordlist_title[word] += 1
            else:
                wordlist_title[word] = 1
        #if out == True:
        #    out = False
        #    break
    #break
    print(total)
wordlist_title

Cell_Phones_and_Accessories.pkl
282198
Office_Products.pkl
417298
Movies_and_TV.pkl
417351
Patio_Lawn_and_Garden.pkl
529729
Clothing_Shoes_and_Jewelry.pkl
2043156
Electronics.pkl
2463881
Industrial_and_Scientific.pkl
2489700
Kindle_Store.pkl
2489700
Digital_Music.pkl
2489723
Home_and_Kitchen.pkl
3026039
Sports_and_Outdoors.pkl
3391802
CDs_and_Vinyl.pkl
3391873
Books.pkl
3392839
Arts_Crafts_and_Sewing.pkl
3505092
Toys_and_Games.pkl
3747531
Grocery_and_Gourmet_Food.pkl
3755740
AMAZON_FASHION.pkl
3764050
Pet_Supplies.pkl
3872506
Automotive.pkl
4291352
Tools_and_Home_Improvement.pkl
4527550


{}

In [89]:
'barnes' in words.words()

False

In [116]:
keyList = list(wordlist.keys())
print(len(keyList))

845142


In [117]:
for key in keyList:
    if wordlist[key] <= 30:
        del wordlist[key]

In [118]:
keyList = list(wordlist.keys())
print(len(keyList))

71303


In [119]:
count = 0
for key in keyList:
    count += 1
    if count % 200 == 0:
        print(count)
    if not key in words.words():
        del wordlist[key]

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800
31000
31200
31400
31600
31800
32000
32200
32400
32600
32800
33000
33200
33400
33600
33800
34000
34200
34400
34600
34800
35000
352

In [123]:
import pandas as pd
wordListFile = pd.DataFrame.from_dict(wordlist, orient='index')

In [124]:
wordListFile = wordListFile.reset_index().rename(columns={'index': 'word', 0: 'count'})
wordListFile = wordListFile.sort_values(by=['word'], ignore_index=True)
wordListFile = wordListFile[wordListFile['count'] > 30]
wordListFile

,word,count
0,a,14740565
1,aa,79082
2,aam,31
3,aba,1370
4,abaca,116
...,...,...
21469,zoom,32406
21470,zooplankton,40
21471,zorro,140
21472,zucchini,701


In [125]:
wordListFile.to_csv('WordList.csv', index=False)

In [126]:
wordListFile.head(60)

,word,count
0,a,14740565
1,aa,79082
2,aam,31
3,aba,1370
4,abaca,116
5,abacus,602
6,abalone,1735
7,abandon,99
8,abandoned,85
9,abas,34


In [75]:
print(wordListFile['count'].sum())
wordListFile.to_csv('WordList.csv')

422130557


## 'asin','title','feature','imageURLHighRes'

In [9]:
# 依据条件筛选数据
filenames = os.listdir(r'/data/liufengyuan/NLPinFinance/Data')
total = 0
for name in filenames:

    data = {}
    filename_path = os.path.join(r'/data/liufengyuan/NLPinFinance/Data',name)
    category = re.findall('meta_(.*?).json.gz',name)[0]
    data[category] = []

    with gzip.open(filename_path) as f:

        count = 0
        for l in f:
            count += 1
            pre_data = json.loads(l.strip())
            if 'imageURLHighRes' in pre_data.keys() and 'feature' in pre_data.keys() and 'description' in pre_data.keys() and 'title' in pre_data.keys() and 'asin' in pre_data.keys():
                if len(pre_data['imageURLHighRes']) != 0 and len(pre_data['description']) != 0 and len(pre_data['feature']) != 0 and len(pre_data['title']) != 0 and len(pre_data['asin']) != 0:
                    data[category].append({k : pre_data[k] for k in ('asin','title','feature', 'description', 'imageURLHighRes')})

    print(f'{category}中满足条件的共{len(data[category])}条,占比{round(100*len(data[category])/count,2)}%')
    total += len(data[category])
    # 字典保存
    path = os.path.join(r'/data/liufengyuan/NLPinFinance/Filtered Data',category)
    f_save = open(path + '.pkl', 'wb')
    pickle.dump(data[category], f_save)
    f_save.close() # 1276549
print(total)

Tools_and_Home_Improvement中满足条件的共236198条,占比41.33%
AMAZON_FASHION中满足条件的共8310条,占比4.45%
Office_Products中满足条件的共135100条,占比42.83%
Pet_Supplies中满足条件的共108456条,占比52.65%
Industrial_and_Scientific中满足条件的共25819条,占比15.42%
Toys_and_Games中满足条件的共242439条,占比38.25%
Arts_Crafts_and_Sewing中满足条件的共112253条,占比37.05%
Kindle_Store中满足条件的共0条,占比0.0%
CDs_and_Vinyl中满足条件的共71条,占比0.01%
Sports_and_Outdoors中满足条件的共365763条,占比38.01%
Automotive中满足条件的共418846条,占比44.97%
Cell_Phones_and_Accessories中满足条件的共282198条,占比47.82%
Books中满足条件的共966条,占比0.03%
Patio_Lawn_and_Garden中满足条件的共112378条,占比40.21%
Home_and_Kitchen中满足条件的共536316条,占比41.24%
Clothing_Shoes_and_Jewelry中满足条件的共1513427条,占比56.36%
Electronics中满足条件的共420725条,占比53.5%
Movies_and_TV中满足条件的共53条,占比0.03%
Grocery_and_Gourmet_Food中满足条件的共8209条,占比2.86%
Digital_Music中满足条件的共23条,占比0.03%
4527550
